In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras import losses
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
from sklearn.metrics import log_loss
from glob import glob
import csv  

2022-03-28 21:26:51.799919: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 21:26:51.799940: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:

path = '/home/dynaneurotech/Documents/kaggle/input/homework4/birds_400/train/'
train_datagen = ImageDataGenerator(rescale=1. / 224)
train = train_datagen.flow_from_directory(path, target_size=(224,224), class_mode='categorical')

classes = [x[0].split('/')[-1] for x in os.walk(path)]
classes.pop(0)

Found 58388 images belonging to 400 classes.


''

In [7]:
# Using alexnet
def AlexNet(input_shape):
    
    input = Input(input_shape)
    
    output = Conv2D(96,(11,11),strides = 4,name="conv0")(input)
    output = BatchNormalization(axis = 3 , name = "bn0")(output)
    output = Activation('relu')(output)
    
    output = MaxPooling2D((3,3),strides = 2,name = 'max0')(output)
    
    output = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(output)
    output = BatchNormalization(axis = 3 ,name='bn1')(output)
    output = Activation('relu')(output)
    
    output = MaxPooling2D((3,3),strides = 2,name = 'max1')(output)
    
    output = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(output)
    output = BatchNormalization(axis = 3, name = 'bn2')(output)
    output = Activation('relu')(output)
    
    output = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(output)
    output = BatchNormalization(axis = 3, name = 'bn3')(output)
    output = Activation('relu')(output)
    
    output = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(output)
    output = BatchNormalization(axis = 3, name = 'bn4')(output)
    output = Activation('relu')(output)
    
    output = MaxPooling2D((3,3),strides = 2,name = 'max2')(output)
    
    output = Flatten()(output)
    
    output = Dense(4096, activation = 'relu', name = "fc0")(output)
    
    output = Dense(4096, activation = 'relu', name = 'fc1')(output) 
    
    output = Dense(400,activation='softmax',name = 'fc2')(output)
    model = Model(inputs = input, outputs = output, name='AlexNet')
    
    return model

In [9]:
model_alex = AlexNet(train[0][0].shape[1:])

In [10]:
# model summary because keras has that
model_alex.summary()

Model: "AlexNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv0 (Conv2D)              (None, 54, 54, 96)        34944     
                                                                 
 bn0 (BatchNormalization)    (None, 54, 54, 96)        384       
                                                                 
 activation_10 (Activation)  (None, 54, 54, 96)        0         
                                                                 
 max0 (MaxPooling2D)         (None, 26, 26, 96)        0         
                                                                 
 conv1 (Conv2D)              (None, 26, 26, 256)       614656    
                                                                 
 bn1 (BatchNormalization)    (None, 26, 26, 256)       1024

In [11]:
model_alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [14]:
model_alex.fit_generator(train,epochs=3)

/tmp/ipykernel_196443/3169902923.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_alex.fit_generator(train,epochs=3)


Epoch 1/3
 612/1825 [=========>....................] - ETA: 20:51 - loss: 5.7758 - accuracy: 0.0073

In [ ]:
path_test = '/home/dynaneurotech/Documents/kaggle/input/homework4/birds_400/test'
test_datagen = ImageDataGenerator(rescale=1. / 224)
test = test_datagen.flow_from_directory(path_test, target_size=(224,224), class_mode='categorical')

In [ ]:
preds = model_alex.evaluate_generator(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100))

In [ ]:
#validation
path_test = '/home/dynaneurotech/Documents/kaggle/input/homework4/birds_400/valid'
validation_datagen = ImageDataGenerator(rescale=1. / 224)
validation = validation_datagen.flow_from_directory(path_test, target_size=(224,224), batch_size = 1,class_mode='categorical')

In [ ]:
preds = model_alex.evaluate_generator(validation)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100))

In [ ]:
def prepare(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = x/224
    return np.expand_dims(x, axis=0)

In [ ]:
cce = losses.CategoricalCrossentropy()

In [ ]:
template = [0]*400
log_loss = {}
test_images = glob("/kaggle/input/csc4851-homework4/birds_400/test/" + "*/*.jpg")

for path in test_images:
    result = model_alex.predict([prepare(path)])
    actual_class = path.split('/')[-2]
    actual_class = "BLACK & YELLOW  BROADBILL" if actual_class == "BLACK & YELLOW BROADBILL" else actual_class
    actual_index = classes.index(actual_class)
    template[actual_index] = 1
    log_loss_current = cce(template, result[0]).numpy()
#     print(log_loss_current)
    if actual_index in log_loss:
        log_loss[actual_index] += (log_loss_current)/100
    else:
        log_loss[actual_index] = (log_loss_current)/100
    template[actual_index] = 0

In [ ]:
print('Generating submission.csv file...')
ids = list(log_loss.keys())
values = list(log_loss.values())

f = open('submission.csv', 'w')
writer = csv.writer(f)
writer.writerow(['id','birds'])
for index in range(len(ids)):
    writer.writerow([ids[index],values[index]])
